In [8]:
# Standard libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Handeling .wav files

import librosa
from librosa import feature

# Machine Learning

from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold

from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from functools import reduce

# data vizualisation

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
def merge_pd(machine):
    
    # Opening the .csv files
    
    df_6dB=pd.read_csv(f'Librosa_features_{machine}_6dB.csv')
    df_0dB=pd.read_csv(f'Librosa_features_{machine}_0dB.csv')
    df_min6dB=pd.read_csv(f'Librosa_features_{machine}_-6dB.csv')
    
    # Dropping the 'Unnamed: 0' column
    
    df_6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_0dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_min6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    
    # Add the number of dB to each column name for each .csv file

    df_6dB.columns = [str(col) + '_6dB' for col in df_6dB.columns]
    df_0dB.columns = [str(col) + '_0dB' for col in df_0dB.columns]
    df_min6dB.columns = [str(col) + '_-6dB' for col in df_min6dB.columns]
    
    # Merging the .csv files into one DataFrame
    
    data_frames = [df_6dB, df_0dB, df_min6dB]
    df_merged = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,how='outer'), data_frames)
    
    df_merged.head()
    
    return df_merged

In [10]:
slider = merge_pd('fan')

In [11]:
slider.drop(columns=['normal(0)/abnormal(1)_6dB','normal(0)/abnormal(1)_0dB'],axis=1,inplace=True)
slider.rename(columns={'normal(0)/abnormal(1)_-6dB': 'normal(0)/abnormal(1)'}, inplace=True)

In [12]:
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', SVC())])

In [13]:
# 20% of the overal data will seperated for later validation of the model

X_model, X_valid = train_test_split(slider,test_size=0.2,random_state = 42)

y_valid = X_valid['normal(0)/abnormal(1)']
X_valid = X_valid.drop(columns=['normal(0)/abnormal(1)'])

# 60% of the overal data (75% of X_model, y_model) will be used to create a training set for the model
# 20% of the overal data (25% of X_model, y_model) will be used to create a testing set for the model

X_train, X_test, y_train, y_test = train_test_split(X_model.drop(columns=['normal(0)/abnormal(1)'], axis=1),
                                                   X_model['normal(0)/abnormal(1)'],
                                                   test_size = 0.25,
                                                   random_state = 10)

In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 56em; }</style>"))

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.feature_selection import RFE

y_pred = []
clfs = []

clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
clfs.append(RandomForestClassifier(class_weight='balanced'))
#clfs.append(EasyEnsembleClassifier(importance_getter=index))

#feature_list = [6,13,21]

#for index in feature_list:
for index in range(4,22):
    
    for classifier in clfs:
        
        print("----------------------------------------------")
        print("----------------------------------------------")
        print(classifier)
        print("----------------------------------------------")
        print("----------------------------------------------")
        
        sel = RFE(classifier, n_features_to_select = index)
        sel.fit(X_train, y_train)
        features = X_train.columns[sel.get_support()]
        X_train_rfe = sel.transform(X_train)
        X_test_rfe = sel.transform(X_test)
        print('Selected Feature', index)
        print(features)
    
        classifier.fit(X_train_rfe, y_train)
        y_pred= classifier.predict(X_test_rfe)
        scores = cross_val_score(pipeline, X_train_rfe, y_train, cv=5)


        print("----------------------------------------------")
        print("TRAIN-TEST")
        print("----------------------------------------------")


        print('confusion matrix', classifier)
        print(confusion_matrix(y_test, y_pred))
        print('classification report')
        print(classification_report(y_test, y_pred))
        print('accuracy score')
        print(accuracy_score(y_test, y_pred))
        
        X_valid_rfe = sel.transform(X_valid)
        y_pred = classifier.predict(X_valid_rfe)

        print("----------------------------------------------")
        print("TRAIN-VALIDATION")
        print("----------------------------------------------")

        print('confusion matrix', classifier)
        print(confusion_matrix(y_valid, y_pred))
        print('classification report')
        print(classification_report(y_valid, y_pred))
        print('accuracy score')
        print(accuracy_score(y_valid, y_pred))

----------------------------------------------
----------------------------------------------
DecisionTreeClassifier()
----------------------------------------------
----------------------------------------------
Selected Feature 4
Index(['melspectrogram_6dB', 'melspectrogram_std_6dB', 'rms_6dB',
       'spectral_centroid_6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[815  10]
 [ 12 273]]
classification report
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       825
           1       0.96      0.96      0.96       285

    accuracy                           0.98      1110
   macro avg       0.98      0.97      0.97      1110
weighted avg       0.98      0.98      0.98      1110

accuracy score
0.9801801801801802
----------------------------------------------
TRAIN-VALIDATION
------------------------------

Selected Feature 6
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_std_6dB', 'mfcc_6dB', 'rms_6dB',
       'zero_crossing_rate_6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[816   9]
 [ 12 273]]
classification report
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       825
           1       0.97      0.96      0.96       285

    accuracy                           0.98      1110
   macro avg       0.98      0.97      0.98      1110
weighted avg       0.98      0.98      0.98      1110

accuracy score
0.981081081081081
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[806   6]
 [  9 289]]
classification report
              precision    recall  f1-score   support

         

Selected Feature 8
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_std_6dB', 'mfcc_6dB',
       'rms_6dB', 'spectral_centroid_6dB', 'min harm_6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[815  10]
 [ 10 275]]
classification report
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       825
           1       0.96      0.96      0.96       285

    accuracy                           0.98      1110
   macro avg       0.98      0.98      0.98      1110
weighted avg       0.98      0.98      0.98      1110

accuracy score
0.9819819819819819
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[803   9]
 [ 12 286]]
classification report
              precisio

Selected Feature 10
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_std_6dB', 'mfcc_6dB',
       'rms_6dB', 'zero_crossing_rate_6dB', 'max harm_6dB', 'max perc_6dB',
       'min harm_6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[808  17]
 [ 12 273]]
classification report
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       825
           1       0.94      0.96      0.95       285

    accuracy                           0.97      1110
   macro avg       0.96      0.97      0.97      1110
weighted avg       0.97      0.97      0.97      1110

accuracy score
0.9738738738738739
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[798  14]
 [ 12 286]]
cla

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[822   3]
 [ 10 275]]
classification report
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       825
           1       0.99      0.96      0.98       285

    accuracy                           0.99      1110
   macro avg       0.99      0.98      0.98      1110
weighted avg       0.99      0.99      0.99      1110

accuracy score
0.9882882882882883
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[810   2]
 [  6 292]]
classification report
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       812
           1       0.99      0.98      0.99       298

    accuracy                           

...

Selected Feature 15
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_corr_6dB',
       'melspectrogram_std_6dB', 'mfcc_6dB', 'rms_6dB',
       'spectral_contrast_6dB', 'zero_crossing_rate_6dB', 'max harm_6dB',
       'max perc_6dB', 'min harm_6dB', 'mfcc_0dB', 'rms_0dB', 'mean perc_0dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[808  17]
 [ 16 269]]
classification report
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       825
           1       0.94      0.94      0.94       285

    accuracy                           0.97      1110
   macro avg       0.96      0.96      0.96      1110
weighted avg       0.97      0.97      0.97      1110

accuracy score
0.9702702702702702
----------------------------------------------
TRAIN-VALIDATION
----------------

Selected Feature 16
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_std_6dB', 'mfcc_6dB',
       'rms_6dB', 'spectral_centroid_6dB', 'spectral_contrast_6dB',
       'spectral_flatness _6dB', 'spectral_rolloff_6dB',
       'zero_crossing_rate_6dB', 'max harm_6dB', 'min harm_6dB',
       'min perc_6dB', 'melspectrogram_std_0dB', 'rms_0dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[824   1]
 [ 10 275]]
classification report
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       825
           1       1.00      0.96      0.98       285

    accuracy                           0.99      1110
   macro avg       0.99      0.98      0.99      1110
weighted avg       0.99      0.99      0.99      1110

accuracy score
0.9900900900900901
-----

Selected Feature 18
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_corr_6dB',
       'melspectrogram_std_6dB', 'mfcc_6dB', 'rms_6dB',
       'spectral_centroid_6dB', 'spectral_bandwidth_6dB',
       'spectral_flatness _6dB', 'spectral_rolloff_6dB',
       'zero_crossing_rate_6dB', 'max harm_6dB', 'min harm_6dB',
       'min perc_6dB', 'melspectrogram_sum_0dB', 'melspectrogram_std_0dB',
       'rms_0dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier()
[[823   2]
 [ 10 275]]
classification report
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       825
           1       0.99      0.96      0.98       285

    accuracy                           0.99      1110
   macro avg       0.99      0.98      0.99      1110
weighted avg       0.99      0.99      0.99      

Selected Feature 20
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_corr_6dB',
       'melspectrogram_std_6dB', 'mfcc_6dB', 'rms_6dB',
       'spectral_centroid_6dB', 'spectral_contrast_6dB',
       'spectral_rolloff_6dB', 'max harm_6dB', 'max perc_6dB', 'min harm_6dB',
       'min perc_6dB', 'mfcc_0dB', 'rms_0dB', 'mean harm_0dB', 'mean perc_0dB',
       'min perc_0dB', 'min perc_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[809  16]
 [ 17 268]]
classification report
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       825
           1       0.94      0.94      0.94       285

    accuracy                           0.97      1110
   macro avg       0.96      0.96      0.96      1110
weighted avg       0.97      0.97      0.97      1110

accuracy sc

Selected Feature 21
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_std_6dB', 'mfcc_6dB',
       'rms_6dB', 'spectral_centroid_6dB', 'spectral_bandwidth_6dB',
       'spectral_contrast_6dB', 'spectral_flatness _6dB',
       'spectral_rolloff_6dB', 'zero_crossing_rate_6dB', 'max harm_6dB',
       'max perc_6dB', 'min harm_6dB', 'min perc_6dB', 'melspectrogram_0dB',
       'melspectrogram_sum_0dB', 'melspectrogram_std_0dB', 'mfcc_0dB',
       'rms_0dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[825   0]
 [ 11 274]]
classification report
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       825
           1       1.00      0.96      0.98       285

    accuracy                           0.99      1110
   macro avg       0.99      0.9